In [20]:
import numpy as np
import pandas as pd
import math
import warnings

warnings.simplefilter("ignore", DeprecationWarning)

In [24]:
case = pd.read_csv("data/data/covid_19.csv")
hos1 = pd.read_csv("data/data/hospital1.csv").iloc[:, 1:]
hos2 = pd.read_csv("data/data/hospital2.csv")

In [4]:
case["submission_date"] = case["submission_date"].astype("datetime64")
case = case.loc[:, ["submission_date", "state", "tot_cases", "new_case"]]
case["lag_date"] = case["submission_date"].shift(1)

In [5]:
join = pd.merge(case, case, how="left", left_on = ["submission_date", "state"], right_on = ["lag_date", "state"])
join["inc_rate"] = join["new_case_y"] / join["tot_cases_x"] 
join = join.fillna(0)

In [15]:
def helper1(state):
    tmp = join[join["state"] == state].reset_index(drop=True)
    inf_ind = np.where(tmp.inc_rate==math.inf)[0]
    return tmp.loc[inf_ind[0]+1:, ["submission_date_x", "state", "inc_rate"]] if inf_ind else tmp.loc[:, ["submission_date_x", "state", "inc_rate"]]
    
case_ = pd.concat([helper1(x) for x in np.unique(join["state"])],axis=0).reset_index(drop=True)

In [12]:
case_.columns = ["date", "state", "inc_rate"]
#case_.merge(hos1, how='left', on="state")

In [14]:
case_

,date,state,inc_rate
0,2020-03-13,AK,0.000000
1,2020-03-14,AK,0.000000
2,2020-03-15,AK,0.000000
3,2020-03-16,AK,2.000000
4,2020-03-17,AK,1.000000
...,...,...,...
14172,2020-10-19,WY,0.023091
14173,2020-10-20,WY,0.033802
14174,2020-10-21,WY,0.027518
14175,2020-10-22,WY,0.042099


In [25]:
def helper2(state, shift=5):
    tmp = case_[case_["state"] == state]
    for i in range(1, shift+1):
        varname = "lag_" + str(i)
        tmp[varname] = tmp["inc_rate"].shift(i)
    return tmp

pd.concat([helper2(x) for x in np.unique(join["state"])],axis=0).reset_index(drop=True).merge(hos1, how='left', on="state")

C:\Users\cnyin\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,submission_date_x,state,inc_rate,lag_1,lag_2,lag_3,lag_4,lag_5,inpatient_beds,inpatient_beds_coverage,...,inpatient_bed_covid_utilization_numerator,inpatient_bed_covid_utilization_denominator,adult_icu_bed_covid_utilization,adult_icu_bed_covid_utilization_coverage,adult_icu_bed_covid_utilization_numerator,adult_icu_bed_covid_utilization_denominator,adult_icu_bed_utilization,adult_icu_bed_utilization_coverage,adult_icu_bed_utilization_numerator,adult_icu_bed_utilization_denominator
0,2020-03-13,AK,0.000000,NaN,NaN,NaN,NaN,NaN,1491.0,28.0,...,32.0,1491.0,0.139535,7.0,6.0,43.0,0.488889,9.0,22.0,45.0
1,2020-03-14,AK,0.000000,0.000000,NaN,NaN,NaN,NaN,1491.0,28.0,...,32.0,1491.0,0.139535,7.0,6.0,43.0,0.488889,9.0,22.0,45.0
2,2020-03-15,AK,0.000000,0.000000,0.000000,NaN,NaN,NaN,1491.0,28.0,...,32.0,1491.0,0.139535,7.0,6.0,43.0,0.488889,9.0,22.0,45.0
3,2020-03-16,AK,2.000000,0.000000,0.000000,0.000000,NaN,NaN,1491.0,28.0,...,32.0,1491.0,0.139535,7.0,6.0,43.0,0.488889,9.0,22.0,45.0
4,2020-03-17,AK,1.000000,2.000000,0.000000,0.000000,0.000000,NaN,1491.0,28.0,...,32.0,1491.0,0.139535,7.0,6.0,43.0,0.488889,9.0,22.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14172,2020-10-19,WY,0.023091,0.031690,0.023707,0.017426,0.034627,0.024214,1943.0,33.0,...,35.0,1943.0,0.144444,29.0,13.0,90.0,0.495050,30.0,50.0,101.0
14173,2020-10-20,WY,0.033802,0.023091,0.031690,0.023707,0.017426,0.034627,1943.0,33.0,...,35.0,1943.0,0.144444,29.0,13.0,90.0,0.495050,30.0,50.0,101.0
14174,2020-10-21,WY,0.027518,0.033802,0.023091,0.031690,0.023707,0.017426,1943.0,33.0,...,35.0,1943.0,0.144444,29.0,13.0,90.0,0.495050,30.0,50.0,101.0
14175,2020-10-22,WY,0.042099,0.027518,0.033802,0.023091,0.031690,0.023707,1943.0,33.0,...,35.0,1943.0,0.144444,29.0,13.0,90.0,0.495050,30.0,50.0,101.0


In [28]:
hos1.describe().to_csv("hos1_des.csv")